This file runs code and analysis for our extensions

In [1]:
# Imports
from mimic_constants import *


In [2]:
# loading master non-idf dataframe
df_master = pd.read_pickle(feature_folder / 'MIMIC_features_v3.pkl')
df_master.shape

(22982, 102)

Prepping Data (copying preprocessing method done in barttender/notebooks/mimic-lr-no-idp.ipynb)

In [3]:
# Labeling columns
indexing_cols = ['subject_id', 'study_id']
imaging_cols  = ['ViewPosition', 'path']
icu_cols = ['hadm_id', 'stay_id', 'first_careunit', 'last_careunit', 'intime', 'outtime', 'los', 'Match',\
    'EarlyBoundary', 'PostGapStart', 'PostGapStop']
label_cols = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', \
    'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',\
    'Fracture', 'Support Devices']
demographic_cols = ['ethnicity', 'anchor_age', 'anchor_year', 'gender']
chart_labels_mean_cols = list(chart_labels_mean.values())
chart_labels_max_cols  = list(chart_labels_max.values())
chart_labels_min_cols  = list(chart_labels_min.values())
lab_labels_mean_cols   = list(lab_labels_mean.values())
lab_labels_max_cols    = list(lab_labels_max.values())
lab_labels_min_cols    = list(lab_labels_min.values())

In [4]:
# removing columns with too many NaNs
df = df_master
select_col_groups = []
removed_col_groups = []
og_col_groups = [chart_labels_mean_cols, chart_labels_max_cols, chart_labels_min_cols, lab_labels_mean_cols, lab_labels_max_cols, lab_labels_min_cols]
for col_group in og_col_groups:
    nan_counts = df_master[col_group].isna().sum()
    too_many_nans = nan_counts[nan_counts > 1000].index.tolist()
    df = df.drop(too_many_nans, axis=1)
    col_group = list(set(col_group) - set(too_many_nans))
    select_col_groups.append(col_group)
    removed_col_groups.append(too_many_nans)

In [5]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix

label = 'Cardiomegaly'
df = df[df[label].isin([0, 1])]  # pick only positive and negative samples from label
f'Number of Total Samples: {len(df)}'

'Number of Total Samples: 11423'

In [6]:
study_year = np.floor(df['StudyDate'] / 10000)
delta_years = study_year - df['anchor_year']
df['age'] = df['anchor_age'] + delta_years
df['age_label'] = df['age'] / 100

df['age_label'] = df['age'] / 100

df['race_label'] = df['ethnicity']
df.loc[df['race_label'] == 'White', 'race_label'] = 0
df.loc[df['race_label'] == 'Asian', 'race_label'] = 1
df.loc[df['race_label'] == 'Black', 'race_label'] = 2
df.loc[df['race_label'] == 'Hispanic/Latino', 'race_label'] = 3
df.loc[df['race_label'] == 'Other', 'race_label'] = 4

df['sex_label'] = df['gender']
df.loc[df['sex_label'] == 'M', 'sex_label'] = 0
df.loc[df['sex_label'] == 'F', 'sex_label'] = 1

demographic_cols = ['age_label', 'race_label', 'sex_label']

X = df[['split'] + demographic_cols + select_col_groups[0] + select_col_groups[3]]
Y = df[['split'] + [label]]
X_train = X[X['split'] == 'train'].drop(['split'], axis=1)
Y_train = Y[Y['split'] == 'train'].drop(['split'], axis=1)[label].astype(float)
X_test = X[X['split'] == 'test'].drop(['split'], axis=1)
Y_test = Y[Y['split'] == 'test'].drop(['split'], axis=1)[label].astype(float)

In [7]:
# 1. Adjust Train-Test Split to 90/10
# Calculate the number of samples to move from train to test
additional_test_samples = 916

# Sampling additional samples from X_train and Y_train
X_train_to_test, X_train = train_test_split(X_train, test_size=(11110 - additional_test_samples) / 11110, random_state=42, stratify=Y_train)
Y_train_to_test, Y_train = train_test_split(Y_train, test_size=(11110 - additional_test_samples) / 11110, random_state=42, stratify=Y_train)

# Concatenate the sampled data to the test sets
X_test = pd.concat([X_test, X_train_to_test], axis=0)
Y_test = pd.concat([Y_test, Y_train_to_test], axis=0)

In [8]:
# 2. Mean Imputation for NaNs in X_train
imputer = SimpleImputer(strategy='mean')
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test_imputed = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns)

# 3. One-Hot Encoding for 'race_label'
X_train_encoded = pd.get_dummies(X_train_imputed, columns=['race_label'], drop_first=True, dtype=float)
X_test_encoded = pd.get_dummies(X_test_imputed, columns=['race_label'], drop_first=True, dtype=float)

# Ensure that the train and test sets have the same columns after encoding
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

X_train_encoded = sm.add_constant(X_train_encoded)  # Add intercept term
X_test_encoded = sm.add_constant(X_test_encoded)

X_train_encoded = X_train_encoded.reset_index(drop=True)
X_test_encoded = X_test_encoded.reset_index(drop=True)
Y_train = Y_train.reset_index(drop=True)
Y_test = Y_test.reset_index(drop=True)

Replicating original paper's logistic regression method to get model metrics

In [11]:
# Copying LR model from barttender/notebooks/mimic-lr-no-idp.ipynb
# 4. Logistic Regression using Statsmodels
logit_model = sm.Logit(Y_train, X_train_encoded)
result = logit_model.fit()

# Predictions and evaluations
Y_test_pred_prob = result.predict(X_test_encoded)
Y_test_pred = (Y_test_pred_prob > 0.5).astype(int)

auc_score = roc_auc_score(Y_test, Y_test_pred_prob)
f1 = f1_score(Y_test, Y_test_pred)
conf_matrix = confusion_matrix(Y_test, Y_test_pred)

# Output
print(f'AUC: {auc_score}', f'F1 Score: {f1}', f'Confusion Matrix:\n{conf_matrix}', sep='\n')

Optimization terminated successfully.
         Current function value: 0.622207
         Iterations 11
AUC: 0.7190507663739653
F1 Score: 0.5647840531561462
Confusion Matrix:
[[485 138]
 [255 255]]


EXTENSION: LASSO Regression as alternate variable selection method

In [14]:
# Fitting LassoCV model, best lambda (alpha) coefficient is found via 10-fold cross-validation
from sklearn.linear_model import LassoCV
lasso_cv = LassoCV(cv=10)
lasso_cv.fit(X_train_encoded, Y_train)
print("Best lambda (alpha):", lasso_cv.alpha_)

Best lambda (alpha): 0.050026338306222065


In [10]:
# predictions and evaluations
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix
Y_pred_prob = lasso_cv.predict(X_test_encoded)
Y_pred = (Y_pred_prob > 0.5).astype(int)

auc_score = roc_auc_score(Y_test, Y_pred_prob)
f1 = f1_score(Y_test, Y_pred)
conf_matrix = confusion_matrix(Y_test, Y_pred)
print(f'AUC: {auc_score}', f'F1 Score: {f1}', f'Confusion Matrix:\n{conf_matrix}', sep='\n')

AUC: 0.7036886664778271
F1 Score: 0.4713216957605985
Confusion Matrix:
[[520 103]
 [321 189]]


Copy of AUC, F1 Score, and Confusion Matrix results from Logistic Regression in main barttender experiment (barttender/notebooks/mimic-lr-no-idp.ipynb):

AUC: 0.7190507663739653
F1 Score: 0.5647840531561462
Confusion Matrix:
[[485 138]
 [255 255]]

LASSO Feature Selection

In [11]:
# Finding which features have nonzero coefficients (indicating significance)
nonzero_indices = np.where(lasso_cv.coef_ != 0)[0]
X_train_encoded.columns[nonzero_indices]

Index(['NBPd_mean', 'SpO2_mean', 'Temp(F)_mean', 'Sodium_mean',
       'PlateletCount_mean', 'RR_mean', 'NBPm_mean', 'HR_mean', 'Glucose_mean',
       'Urea_Nitrogren_mean', 'Hematocrit_mean', 'Chloride_mean'],
      dtype='object')

In [12]:
# Features selected by LASSO:
lasso_significant_variables = ['NBPd_mean', 'Temp(F)_mean', 'NBPm_mean', 'Sodium_mean',
       'PlateletCount_mean', 'RR_mean', 'HR_mean', 'SpO2_mean',
       'Hematocrit_mean', 'Chloride_mean', 'Urea_Nitrogren_mean',
       'Glucose_mean']


Recreating main Barttender experiment (Images + bars) with new significant variables

In [13]:
# Rewriting functions from mimic_constants.py, replacing default bar_variables values with lasso_significant_variables
def get_barcode_order_info(order=None, no_bars=False, nan=False, only_bars=False, bar_variables=lasso_significant_variables):
    if no_bars:
        return None, 'no_bars'
    if nan:
        bar_variables=lasso_significant_variables
    if only_bars:
        bar_variables=lasso_significant_variables
    og_order = np.array(bar_variables)
    order = [int(x) for x in order.strip('[]').split(',')] if isinstance(order, str) else order
    order = list(range(len(bar_variables))) if order is None else order
    suffix = '_'.join(og_order[order]).replace('_mean', '').replace('_val', '').lower()
    return order, suffix


def construct_preproc_path(og_path: str, root_dir: Path, preproc_dir=None):
    split = og_path.split("/")
    preproc_filename = f"{split[2]}_{split[3]}_{split[4]}"
    if preproc_dir:
        return root_dir / preproc_dir / preproc_filename
    else:
        return root_dir / preproc_filename
    

In [14]:
# Copying functions from barttender/mimic_add_bar_preprocess.py, replacing default bar_variables values with lasso_significant_variables
import os
import fire
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import QuantileTransformer

from PIL import Image
from skimage.io import imread
from skimage.io import imsave, imshow
from skimage.transform import resize

def npy_bar(data, colormap, img_w=500, img_h=100, add_label=False, add_colormap=False, verbose=False):
    variables = data.columns[1:]  # Exclude ID
    variables = variables[::-1]  # Bars generated bottom to top

    for index, row in data.iterrows():
        px = 1/plt.rcParams['figure.dpi']  # pixel in inches
        bar_height = 1.0  # Height of each bar -- keep it to 1.0 to ensure no gap
        if verbose:
            fig, ax = plt.subplots(figsize=(10, 5))
        else:
            fig, ax = plt.subplots(figsize=(img_w*px, img_h*px))

        # Create horizontal bar plot
        for i, var in enumerate(variables):
            value = row[var]
            if np.isnan(value):
                color = 'r'  # i.e. 'red'
            else:
                color = colormap(value)
            ax.barh(i, 1, color=color, height=bar_height, edgecolor='none')

        # Set y-ticks and labels
        if add_label:
            ax.set_yticks(np.arange(len(variables)))
            ax.set_yticklabels(variables)
        else:
            ax.set_yticks([])

        # Remove grid and axes
        ax.grid(False)
        ax.set_xticks([])
        ax.set_xlim(0, 1)

        # Remove spines
        for spine in ax.spines.values():
            spine.set_visible(False)

        # Add colorbar
        if add_colormap:
            sm = plt.cm.ScalarMappable(
                cmap=colormap, norm=plt.Normalize(vmin=0, vmax=1))
            sm.set_array([])
            cbar = fig.colorbar(sm, ax=ax, fraction=0.025, pad=0.04)
            cbar.set_label('Normalized feature value')

        fig.subplots_adjust(left=0, right=1, top=1, bottom=0)
        fig.canvas.draw()
        if add_label or add_colormap:
            plt.show()
        else:
            bar_img = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
            bar_img = bar_img.reshape(fig.canvas.get_width_height()[::-1] + (4,))[:, :, :3]
            if img_h != 224:
                # slicing removes whitespace from bars; need to redo if bar size is changed
                bar_img = bar_img[3:-2, :, :]  # HWC
            plt.close(fig)
        
            if verbose:
                print(bar_img.shape)
                imshow(bar_img.astype(np.uint8))

            return bar_img

def preprocess_mimic_df(idp=False, order=None, bar_vars=lasso_significant_variables, label='Cardiomegaly', verbose=False):
    variables = np.array(bar_vars)
    order, suffix = get_barcode_order_info(order, bar_vars)
    
    df_master = get_master_df(idp=idp)
    if not idp:
        df_master = df_master[df_master[label].isin([0, 1])]
        study_year = np.floor(df_master['StudyDate'] / 10000)
        delta_years = study_year - df_master['anchor_year']
        df_master['age'] = df_master['anchor_age'] + delta_years
    else:
        df_master['age'] = df_master['anchor_age']

    
    # normalize age as in chexpert (0-100 to 0-1)
    df_master['age_val'] = df_master['age'].apply(lambda x: min(x / 100, 1))
    # quantile normalize the rest
    for var in bar_vars[1:]:
        transformer = QuantileTransformer(output_distribution='uniform')
        df_master.loc[:, var] = transformer.fit_transform(df_master[[var]])

    df = df_master[['path'] + bar_vars]

    if verbose:
        print(df.head())
        print(variables[order].tolist())
    
    return df[['path'] + variables[order].tolist()[0]]

def preprocess_and_append_bars(df, image_type, img_data_dir, idp=False, order=None, bar_vars=lasso_significant_variables, colormap=plt.colormaps['binary'], verbose=False):
    df_cxr = df.copy()
    out_dir = img_data_dir

    order, suffix = get_barcode_order_info(order)
    # Hard coding shorter suffix to reduce file path length and increase clarity
    preproc_dir = get_preproc_subpath(image_type, 'LASSO_Vars')

    print(f'Image Type: {image_type}')
    print(f"Barcode Order: {suffix.replace('_', ', ')}")
    print(f'preproc_dir: {out_dir / preproc_dir}')
    
    bars_h = int(0.2 * 224) # 44
    whitespace_offset = 5 + 1 # num of pixel rows removed from bar output + 1 (row to remove because resizing artifact)

    if not verbose:
        os.makedirs(out_dir/ preproc_dir, exist_ok=True)

    for idx, p in enumerate(tqdm(df_cxr['path'])):
        out_path = construct_preproc_path(p, out_dir, preproc_dir)

        if (not os.path.exists(out_path)) or verbose:
            height = 224 + whitespace_offset - bars_h; width = 224
            if image_type == 'xray':
                image = imread(scratch_dir / p)  # og chexpert imagery is on scratch
                image = resize(image, output_shape=(height, width), preserve_range=True)
                image = np.expand_dims(image, axis=2).repeat(3, axis=2)
            elif image_type == 'noise':
                # Generate random noise
                image = np.random.randint(low=0, high=255, size=(height, width)) # 1-channel, B/W noise
                image = np.expand_dims(image, axis=2).repeat(3, axis=2)  # make it 3-channel
            elif image_type == 'blank':
                image = np.zeros(shape=(height, width, 3))  # all black image

            image = image[:-1,:,:]  # remove last img row (gets rid of resizing artifacts)

            # Generate the barcode and append it to the bottom of the image
            bar_img = npy_bar(df_cxr.iloc[[idx]], colormap, img_h=bars_h, img_w=width)
            
            combined_img = np.concatenate((image, bar_img), axis=0)
            
            if verbose:
                imshow(combined_img.astype(np.uint8))
            else:  
                imsave(out_path, combined_img.astype(np.uint8))

In [26]:
# Preprocess all with new 12 significant features
df_cxr_ex = preprocess_mimic_df(idp=True, verbose=True, bar_vars=lasso_significant_variables)

                                                path  NBPd_mean  Temp(F)_mean  \
0  files/p10/p10001884/s55333410/5b135035-4708084...  74.669683      0.888785   
1  files/p10/p10002013/s52163036/1e647043-eed3576...  64.500000      0.043544   
2  files/p10/p10004457/s55439624/ad41d9ed-37ff140...  52.363636      0.251752   
3  files/p10/p10011365/s56594284/ff295ca5-9f888b5...  56.953271      0.973927   
4  files/p10/p10012292/s52412975/ca16ea39-8c22bfd...  71.602941      0.258071   

   NBPm_mean  Sodium_mean  PlateletCount_mean   RR_mean   HR_mean  SpO2_mean  \
0   0.892855     0.382382            0.297575  0.484323  0.151559   0.726947   
1   0.509147     0.594595            0.770740  0.077055  0.746734   0.510913   
2   0.103103     0.667167            0.272995  0.069201  0.764736   0.853201   
3   0.430352     0.782889            0.528862  0.763676  0.574240   0.026357   
4   0.794877     0.945946            0.514515  0.867930  0.809817   0.040022   

   Hematocrit_mean  Chloride_mea

In [25]:
# Image Barttender processing (xray + bars) 
preprocess_and_append_bars(
    df_cxr_ex,
    image_type='xray',
    img_data_dir=nb_group_dir,
    idp=True,
    bar_vars=lasso_significant_variables,
    verbose=False # IMPORTANT: it won't save the images if you run in verbose mode
)

Image Type: xray
Barcode Order: nbpd, temp(f), nbpm, sodium, plateletcount, rr, hr, spo2, hematocrit, chloride, urea, nitrogren, glucose
preproc_dir: physionet.org\files\mimic-cxr-jpg\2.1.0\preprocessed\preproc_224x224_xray_LASSO_Vars


100%|██████████| 2662/2662 [00:00<00:00, 35025.78it/s]


In [17]:
# Rewriting functions from barttender/mimic_k_fold.py, replacing file path values as needed
import os
import fire
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
import torchvision
import torchvision.transforms as T
from torchvision import models
import pytorch_lightning as pl

from copy import deepcopy

import wandb
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from skimage.io import imread
from skimage.io import imsave
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix, f1_score, matthews_corrcoef

from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.tuner import Tuner
from sklearn.model_selection import KFold

image_size = (224, 224)
num_classes = 1

class MIMICDataset(Dataset):
    def __init__(self, csv_file_img, image_size, img_data_dir: Path, augmentation=False, pseudo_rgb=False):
        self.data = pd.read_csv(csv_file_img)
        self.image_size = image_size
        self.do_augment = augmentation
        self.pseudo_rgb = pseudo_rgb

        self.labels = ['Cardiomegaly']

        self.augment = T.Compose([
            T.RandomHorizontalFlip(p=0.5),
            T.RandomApply(transforms=[T.RandomAffine(degrees=15, scale=(0.9, 1.1))], p=0.5),
        ])

        self.samples = []
        for idx, _ in enumerate(tqdm(range(len(self.data)), desc='Loading Data')):
            og_path = self.data.loc[idx, 'path']
            img_path = construct_preproc_path(og_path, img_data_dir)
            img_label = np.array(self.data.loc[idx, self.labels[0].strip()] == 1, dtype='float32')

            sample = {'image_path': img_path, 'label': img_label}
            self.samples.append(sample)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        sample = self.get_sample(item)

        image = torch.from_numpy(sample['image'])
        # unsqueeze adds an extra dimension to the labels to match the output of the model probs
        label = torch.tensor(sample['label'], dtype=torch.float32).unsqueeze(0)

        if self.do_augment:
            image = self.augment(image)

        if self.pseudo_rgb:
            image = image.unsqueeze(0).repeat(3, 1, 1)
        else:
            image = image.permute(2, 0, 1)

        return {'image': image, 'label': label}

    def get_sample(self, item):
        sample = self.samples[item]
        image = imread(sample['image_path']).astype(np.float32)

        return {'image': image, 'label': sample['label']}

class MIMICDataModule(pl.LightningDataModule):
    def __init__(self, csv_train_img, csv_val_img, csv_test_img, image_size, img_data_dir: Path, pseudo_rgb, batch_size, num_workers):
        super().__init__()
        self.csv_train_img = csv_train_img
        self.csv_val_img = csv_val_img
        self.csv_test_img = csv_test_img
        self.image_size = image_size
        self.batch_size = batch_size
        self.num_workers = num_workers

        self.train_set = MIMICDataset(self.csv_train_img, self.image_size, img_data_dir, augmentation=True, pseudo_rgb=pseudo_rgb)
        self.val_set = MIMICDataset(self.csv_val_img, self.image_size, img_data_dir, augmentation=False, pseudo_rgb=pseudo_rgb)
        self.test_set = MIMICDataset(self.csv_test_img, self.image_size, img_data_dir,  augmentation=False, pseudo_rgb=pseudo_rgb)

        print('#train: ', len(self.train_set))
        print('#val:   ', len(self.val_set))
        print('#test:  ', len(self.test_set))

    def train_dataloader(self):
        return DataLoader(self.train_set, self.batch_size, shuffle=True, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_set, self.batch_size, shuffle=False, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test_set, self.batch_size, shuffle=False, num_workers=self.num_workers)


class ResNet(pl.LightningModule):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes
        self.model = models.resnet34(pretrained=True)
        # freeze_model(self.model)
        num_features = self.model.fc.in_features
        self.model.fc = nn.Linear(num_features, self.num_classes)

    def remove_head(self):
        num_features = self.model.fc.in_features
        id_layer = nn.Identity(num_features)
        self.model.fc = id_layer

    def forward(self, x):
        return self.model.forward(x)

    def configure_optimizers(self):
        params_to_update = []
        for param in self.parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
        optimizer = torch.optim.Adam(params_to_update, lr=0.001)
        return optimizer

    def unpack_batch(self, batch):
        return batch['image'], batch['label']

    def process_batch(self, batch):
        img, lab = self.unpack_batch(batch)
        out = self.forward(img)
        prob = torch.sigmoid(out)
        loss = F.binary_cross_entropy(prob, lab)
        return loss

    def training_step(self, batch, batch_idx):
        loss = self.process_batch(batch)
        self.log('train_loss', loss)
        grid = torchvision.utils.make_grid(batch['image'][0:4, ...], nrow=2, normalize=True)
        self.logger.experiment.add_image('images', grid, self.global_step)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.process_batch(batch)
        self.log('val_loss', loss, sync_dist=True)

    def test_step(self, batch, batch_idx):
        loss = self.process_batch(batch)
        self.log('test_loss', loss)


class DenseNetTrain(pl.LightningModule):
    def __init__(self, num_classes):
        super().__init__()
        self.save_hyperparameters()
        self.num_classes = num_classes
        self.model = models.densenet121(pretrained=True)
        # freeze_model(self.model)
        num_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(num_features, self.num_classes)

    def remove_head(self):
        num_features = self.model.classifier.in_features
        id_layer = nn.Identity(num_features)
        self.model.classifier = id_layer

    def forward(self, x):
        return self.model.forward(x)

    def configure_optimizers(self):
        params_to_update = []
        for param in self.parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
        optimizer = torch.optim.Adam(params_to_update, lr=0.001)
        return optimizer

    def unpack_batch(self, batch):
        return batch['image'], batch['label']

    def process_batch(self, batch):
        img, lab = self.unpack_batch(batch)
        out = self.forward(img)
        prob = torch.sigmoid(out)
        loss = F.binary_cross_entropy(prob, lab)
        return loss

    def training_step(self, batch, batch_idx):
        loss = self.process_batch(batch)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.process_batch(batch)
        self.log('val_loss', loss, sync_dist=True)

    def test_step(self, batch, batch_idx):
        loss = self.process_batch(batch)
        self.log('test_loss', loss)


def freeze_model(model):
    for param in model.parameters():
        param.requires_grad = False


def test(model, data_loader, device):
    model.eval()
    logits = []
    preds = []
    targets = []

    with torch.no_grad():
        for index, batch in enumerate(tqdm(data_loader, desc='Test-loop')):
            img, lab = batch['image'].to(device), batch['label'].to(device)
            out = model(img)
            pred = torch.sigmoid(out)
            logits.append(out)
            preds.append(pred)
            targets.append(lab)

        logits = torch.cat(logits, dim=0)
        preds = torch.cat(preds, dim=0)
        targets = torch.cat(targets, dim=0)

        preds_np = preds.cpu().numpy()
        targets_np = targets.cpu().numpy()

        auc_scores = []
        for i in range(num_classes):
            auc = roc_auc_score(targets_np[:, i], preds_np[:, i])
            auc_scores.append(auc)
        
        for i, auc in enumerate(auc_scores):
            print(f"Cardiomegaly AUC: {auc:.4f}")

    return preds_np, targets_np, logits.cpu().numpy(), auc_scores

def embeddings(model, data_loader, device):
    model.eval()

    embeds = []
    targets = []

    with torch.no_grad():
        for index, batch in enumerate(tqdm(data_loader, desc='Test-loop')):
            img, lab = batch['image'].to(device), batch['label'].to(device)
            emb = model(img)
            embeds.append(emb)
            targets.append(lab)

        embeds = torch.cat(embeds, dim=0)
        targets = torch.cat(targets, dim=0)

    return embeds.cpu().numpy(), targets.cpu().numpy()

def compute_metrics(y_true, y_pred, y_logits):
    """Compute various classification metrics for binary classification."""
    # y_true: Actual binary labels (0 or 1)
    # y_pred: Predicted binary labels (0 or 1), thresholding applied
    # y_logits: Raw model outputs (logits) or probabilities for the positive class

    # Convert y_logits to probabilities if they are logits
    y_prob = y_logits if y_logits.shape[1] == 1 else y_logits[:, 1]

    auc = roc_auc_score(y_true, y_prob)
    avg_precision = average_precision_score(y_true, y_prob)
    
    # Convert y_pred to binary predictions if it's probabilities
    y_pred_binary = (y_pred > 0.5).astype(int)
    
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred_binary)
    
    # Metrics calculations
    tpr = cm[1, 1] / (cm[1, 1] + cm[1, 0]) if (cm[1, 1] + cm[1, 0]) > 0 else 0  # Sensitivity, Recall
    tnr = cm[0, 0] / (cm[0, 0] + cm[0, 1]) if (cm[0, 0] + cm[0, 1]) > 0 else 0  # Specificity
    ppv = cm[1, 1] / (cm[1, 1] + cm[0, 1]) if (cm[1, 1] + cm[0, 1]) > 0 else 0  # Precision
    npv = cm[0, 0] / (cm[0, 0] + cm[1, 0]) if (cm[0, 0] + cm[1, 0]) > 0 else 0  # Negative Predictive Value
    f1 = f1_score(y_true, y_pred_binary)
    mcc = matthews_corrcoef(y_true, y_pred_binary)

    return {
        'AUC': auc,
        'Average Precision': avg_precision,
        'TPR': tpr,
        'TNR': tnr,
        'PPV': ppv,
        'NPV': npv,
        'F1': f1,
        'MCC': mcc
    }


def main(image_type: str = 'xray', order = None, batch_size: int = 192, epochs: int = 100, num_workers: int = 0, n_splits: int = 10, idp: bool = True, nan: bool = False, only_bars: bool = False, no_bars: bool = False):

    # sets seeds for numpy, torch, python.random and PYTHONHASHSEED.
    pl.seed_everything(42, workers=True)
    image_type = 'only_bars' if only_bars else image_type
    order, suffix = get_barcode_order_info(order=order, no_bars=no_bars, nan=nan, only_bars=only_bars)
    preproc_dir = get_preproc_subpath(image_type, "LASSO_Vars")
    root_dir = get_correct_root_dir(preproc_dir)
    img_data_dir = root_dir / preproc_dir

    # Create output directory for this cross-validation experiment
    # Hardcoding - replacing suffix with LASSO_Vars
    exp_dir = home_out_dir / f"cross-val/densenet-{image_type}-{'LASSO_Vars'}-{'idp' if idp else 'no_idp'}/"
    if not os.path.exists(exp_dir):
        os.makedirs(exp_dir)
    df = get_cardiomegaly_df(idp=idp)
    if idp: df['split'] = 'train'
    train_df, val_df, test_df = df_train_test_split(df, test_size=0.2, val_size=0.1)
    train_df.to_csv(exp_dir / 'train.csv')    
    val_df.to_csv(exp_dir / 'val.csv')
    test_df.to_csv(exp_dir / 'test.csv')
    
    data = MIMICDataModule(csv_train_img=(exp_dir / 'train.csv').as_posix(),
                              csv_val_img=(exp_dir / 'val.csv').as_posix(),
                              csv_test_img=(exp_dir / 'test.csv').as_posix(),
                              img_data_dir=img_data_dir,
                              image_size=image_size,
                              pseudo_rgb=False,
                              batch_size=batch_size,
                              num_workers=num_workers)

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Determine GPU usage and training strategy
    use_cuda = torch.cuda.is_available()
    num_gpus = torch.cuda.device_count()
    accelerator = "gpu" if use_cuda else None
    device = "cuda" if use_cuda else "cpu"
    strategy = "ddp" if num_gpus > 1 else "auto"

    # Optimize batch size if only one GPU is available
    if num_gpus == 1 and False:
        model_type = DenseNetTrain(num_classes=num_classes)
        trainer = pl.Trainer(accelerator=accelerator, devices=1)
        tuner = Tuner(trainer)
        optimal_batch_size = tuner.scale_batch_size(model_type, datamodule=data)
        data.batch_size = optimal_batch_size
        print(f'Optimal batch size found: {optimal_batch_size}')
    else:
        optimal_batch_size = batch_size
        print(f'Optimal batch size found: {optimal_batch_size}')

    all_val_metrics = []
    all_test_metrics = []
    fold_metrics = []
    fold_data = deepcopy(data)
    for fold, (train_idx, val_idx) in enumerate(kf.split(data.train_set)):
        print(f'\n--- Fold {fold+1}/{n_splits} ---')

        # Split data for this fold
        train_subset = torch.utils.data.Subset(data.train_set, train_idx)
        val_subset = torch.utils.data.Subset(data.train_set, val_idx)
        
        fold_data.train_set = train_subset
        fold_data.val_set = val_subset

        if max(train_idx) >= len(data.train_set) or max(val_idx) >= len(data.train_set):
            raise IndexError("Indices out of bounds.")

        # Model
        model_type = DenseNetTrain
        model = model_type(num_classes=num_classes)

        checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode='min')
        early_stopping_callback = EarlyStopping(monitor="val_loss", mode='min', patience=5, verbose=True)

        config = {}
        config["pseudo_rgb"] = False
        config["batch_size"] = optimal_batch_size
        config["image_size"] = image_size
        config["image_type"] = image_type
        config["barcode_order"] = suffix
        config["img_data_dir"] = img_data_dir
        config["idp"] = idp
        config['csv_train'] = (exp_dir / 'train.csv').as_posix()
        config['csv_val']   = (exp_dir / 'val.csv').as_posix()
        config['csv_test']  = (exp_dir / 'test.csv').as_posix()

        # Logger for this fold
        wandb_logger = WandbLogger(project='mimic', config=config, save_dir=home_out_dir)

        # Create output directory for this fold
        out_name = f"fold{fold+1}-{wandb_logger.experiment.id}/"
        out_dir = exp_dir / out_name
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)
        wandb_logger.experiment.config["out_dir"] = out_dir

        # Trainer
        trainer = pl.Trainer(
            callbacks=[checkpoint_callback, early_stopping_callback],
            log_every_n_steps=5,
            max_epochs=epochs,
            logger=wandb_logger,
            accelerator=accelerator,
            devices="auto" if use_cuda else 1,
            num_nodes=1,
            strategy=strategy
        )
        trainer.logger._default_hp_metric = False

        # Train the model
        trainer.fit(model, fold_data)

        # Load the best model from this fold
        model = model_type.load_from_checkpoint(trainer.checkpoint_callback.best_model_path, num_classes=num_classes)
        model.to(device)

        # Validation
        print(f'\nVALIDATION for Fold {fold+1}')
        preds_val, targets_val, logits_val, aucs_val = test(model, fold_data.val_dataloader(), device)
        val_metrics = compute_metrics(targets_val, preds_val, logits_val)
        all_val_metrics.append(val_metrics)

        # Testing
        print(f'\nTESTING for Fold {fold+1}')
        preds_test, targets_test, logits_test, aucs_test = test(model, fold_data.test_dataloader(), device)
        test_metrics = compute_metrics(targets_test, preds_test, logits_test)
        all_test_metrics.append(test_metrics)

        # Log metrics for this fold
        for metric_name, metric_value in val_metrics.items():
            wandb_logger.log_metrics({f'fold_{fold+1}_val_{metric_name.lower()}': metric_value})
        for metric_name, metric_value in test_metrics.items():
            wandb_logger.log_metrics({f'fold_{fold+1}_test_{metric_name.lower()}': metric_value})

        cols_names_classes = ['class_' + str(i) for i in range(0,num_classes)]
        cols_names_logits = ['logit_' + str(i) for i in range(0, num_classes)]
        cols_names_targets = ['target_' + str(i) for i in range(0, num_classes)]

        # Save validation and test predictions for this fold
        df_val = pd.DataFrame(data=preds_val, columns=cols_names_classes)
        df_val_logits = pd.DataFrame(data=logits_val, columns=cols_names_logits)
        df_val_targets = pd.DataFrame(data=targets_val, columns=cols_names_targets)
        df_val = pd.concat([df_val, df_val_logits, df_val_targets], axis=1)
        df_val.to_csv(os.path.join(out_dir, 'predictions.val.csv'), index=False)

        df_test = pd.DataFrame(data=preds_test, columns=cols_names_classes)
        df_test_logits = pd.DataFrame(data=logits_test, columns=cols_names_logits)
        df_test_targets = pd.DataFrame(data=targets_test, columns=cols_names_targets)
        df_test = pd.concat([df_test, df_test_logits, df_test_targets], axis=1)
        df_test.to_csv(os.path.join(out_dir, 'predictions.test.csv'), index=False)

        # Store fold-wise metrics
        fold_metrics.append({
            'fold': fold+1,
            'val_metrics': val_metrics,
            'test_metrics': test_metrics
        })

        wandb.finish()

    # Aggregate metrics over all folds
    val_metrics_mean = {key: np.mean([m[key] for m in all_val_metrics]) for key in all_val_metrics[0].keys()}
    test_metrics_mean = {key: np.mean([m[key] for m in all_test_metrics]) for key in all_test_metrics[0].keys()}
    val_metrics_std = {key: np.std([m[key] for m in all_val_metrics]) for key in all_val_metrics[0].keys()}
    test_metrics_std = {key: np.std([m[key] for m in all_test_metrics]) for key in all_test_metrics[0].keys()}
    best_folds = {key: max(enumerate(all_val_metrics), key=lambda x: x[1][key])[0]+1 for key in all_val_metrics[0].keys()}

    print(f'\nMean Validation Metrics across {n_splits} folds: {val_metrics_mean}')
    print(f'\nMean Test Metrics across {n_splits} folds: {test_metrics_mean}')

    # Save overall and fold-wise metrics to CSV
    metrics_summary = {
        'metric': list(val_metrics_mean.keys()),
        'val_mean': list(val_metrics_mean.values()),
        'val_std': list(val_metrics_std.values()),
        'test_mean': list(test_metrics_mean.values()),
        'test_std': list(test_metrics_std.values()),
        'best_fold': [best_folds[key] for key in val_metrics_mean.keys()]
    }
    df_metrics_summary = pd.DataFrame(metrics_summary)
    df_metrics_summary.to_csv(exp_dir / 'metrics_summary.csv', index=False)

    df_fold_metrics = pd.DataFrame(fold_metrics)
    val_metrics_df = df_fold_metrics['val_metrics'].apply(pd.Series).add_prefix('val_')
    test_metrics_df = df_fold_metrics['test_metrics'].apply(pd.Series).add_prefix('test_')

    # Concatenate the original DataFrame with the new metrics columns
    df_fold_metrics = pd.concat([df_fold_metrics[['fold']], val_metrics_df, test_metrics_df], axis=1)
    df_fold_metrics.to_csv(exp_dir / 'fold_metrics.csv', index=False)

    # Log the fold that was best for each metric
    print(f'\nBest folds based on validation metrics: {best_folds}')

Training DenseNet model with LASSO Selected variables

In [ ]:
import importlib
import os
import wandb
os.environ['WANDB_MODE'] = 'offline'

main(
    image_type = 'xray',
    batch_size=32,
    epochs=100,
    n_splits=10,
    idp=True,
    num_workers=0
)

Seed set to 42
Loading Data: 100%|██████████| 537/537 [00:00<00:00, 107407.78it/s]


#train:  1849
#val:    276
#test:   537
Optimal batch size found: 32

--- Fold 1/10 ---


c:\Users\Josh\Desktop\DLH\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Josh\Desktop\DLH\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
wandb: WARNING `resume` will be ignored since W&B syncing is set to `offline`. Starting a new run with run id all2qvtv.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
You are using a CUDA device ('NVIDIA GeForce RTX 4070') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type     | Params | Mode  | FLOPs
---------------------------------------------------
0 | model | DenseNet | 7.0 M  | train | 0    
---------------------------------------------------
7.0 M     Trainable params
0         Non-trainable params
7.0 M     Total params
27.820    Total estimated model params size (MB)
433       Modules in train mode
0         Modules in eval mode
0         Total Flops


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Test-loop: 100%|██████████| 17/17 [00:00<00:00, 17.83it/s]


Cardiomegaly AUC: 0.8719


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇█████
fold_10_test_auc,▁
fold_10_test_average precision,▁
fold_10_test_f1,▁
fold_10_test_mcc,▁
fold_10_test_npv,▁
fold_10_test_ppv,▁
fold_10_test_tnr,▁
fold_10_test_tpr,▁
fold_10_val_auc,▁
+10,...



Mean Validation Metrics across 10 folds: {'AUC': np.float64(0.7905084689243136), 'Average Precision': np.float64(0.8947813411014665), 'TPR': np.float64(0.9080100151091983), 'TNR': np.float64(0.4531920371638535), 'PPV': np.float64(0.7935345054679639), 'NPV': np.float64(0.6808392647037007), 'F1': np.float64(0.8458214058433432), 'MCC': np.float64(0.41206739895242467)}

Mean Test Metrics across 10 folds: {'AUC': np.float64(0.8645590185676392), 'Average Precision': np.float64(0.9335451365883504), 'TPR': np.float64(0.8588859416445624), 'TNR': np.float64(0.675), 'PPV': np.float64(0.8653725500303462), 'NPV': np.float64(0.6783894767540813), 'F1': np.float64(0.860062377384132), 'MCC': np.float64(0.5371901643012171)}

Best folds based on validation metrics: {'AUC': 1, 'Average Precision': 9, 'TPR': 7, 'TNR': 9, 'PPV': 9, 'NPV': 7, 'F1': 4, 'MCC': 9}


## EXTENSION: Mutual Information–based feature selection (BT–MI)

MI - Ranking

In [19]:
from sklearn.feature_selection import mutual_info_classif

# Start from the imputed (not one-hot) features
X_train_mi = X_train_imputed.copy()
X_test_mi  = X_test_imputed.copy()

# Drop any ID-like columns if present
cols_to_drop = ["subject_id", "study_id", "hadm_id", "stay_id", "icustay_id"]
X_train_mi = X_train_mi.drop(columns=[c for c in cols_to_drop if c in X_train_mi.columns], errors="ignore")
X_test_mi  = X_test_mi.drop(columns=[c for c in cols_to_drop if c in X_test_mi.columns], errors="ignore")

mi_scores = mutual_info_classif(
    X_train_mi,
    Y_train,
    discrete_features=False,
    random_state=42,
)

mi_df = (
    pd.DataFrame({"feature": X_train_mi.columns, "mi": mi_scores})
    .sort_values("mi", ascending=False)
)

k = len(lasso_significant_variables)   # keep same number as LASSO
mi_significant_variables = mi_df["feature"].head(k).tolist()

mi_df.head(20)


,feature,mi
19,Creatinine_mean,0.037880
15,Urea_Nitrogren_mean,0.034914
0,age_label,0.032685
14,Phosphate_mean,0.027916
18,Chloride_mean,0.026851
17,Calcium_Total_mean,0.026210
9,RR_mean,0.018079
4,SpO2_mean,0.015815
16,Hematocrit_mean,0.014901
12,Glucose_mean,0.013890


In [20]:
print(f"Top {k} features selected by Mutual Information:")
print("="*60)
for i, feature in enumerate(mi_significant_variables, 1):
    mi_score = mi_df[mi_df['feature'] == feature]['mi'].values[0]
    print(f"{i:2d}. {feature:30s} (MI: {mi_score:.6f})")

print("\n" + "="*60)
print("MI significant variables list:")
print(mi_significant_variables)


Top 12 features selected by Mutual Information:
 1. Creatinine_mean                (MI: 0.037880)
 2. Urea_Nitrogren_mean            (MI: 0.034914)
 3. age_label                      (MI: 0.032685)
 4. Phosphate_mean                 (MI: 0.027916)
 5. Chloride_mean                  (MI: 0.026851)
 6. Calcium_Total_mean             (MI: 0.026210)
 7. RR_mean                        (MI: 0.018079)
 8. SpO2_mean                      (MI: 0.015815)
 9. Hematocrit_mean                (MI: 0.014901)
10. Glucose_mean                   (MI: 0.013890)
11. Magnesium_mean                 (MI: 0.010591)
12. Sodium_mean                    (MI: 0.010566)

MI significant variables list:
['Creatinine_mean', 'Urea_Nitrogren_mean', 'age_label', 'Phosphate_mean', 'Chloride_mean', 'Calcium_Total_mean', 'RR_mean', 'SpO2_mean', 'Hematocrit_mean', 'Glucose_mean', 'Magnesium_mean', 'Sodium_mean']


Logistic regression with CV-tuned C

In [21]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix

# Restrict to MI features
X_train_mi_selected = X_train_imputed[mi_significant_variables]
X_test_mi_selected  = X_test_imputed[mi_significant_variables]

# Scale for better convergence
scaler_mi = StandardScaler()
X_train_mi_scaled = scaler_mi.fit_transform(X_train_mi_selected)
X_test_mi_scaled  = scaler_mi.transform(X_test_mi_selected)

logreg_mi_cv = LogisticRegressionCV(
    Cs=10,
    cv=10,
    scoring="roc_auc",
    max_iter=5000,
    solver="lbfgs"
)

logreg_mi_cv.fit(X_train_mi_scaled, Y_train)

y_prob_mi = logreg_mi_cv.predict_proba(X_test_mi_scaled)[:, 1]
y_pred_mi = (y_prob_mi > 0.5).astype(int)

auc_mi = roc_auc_score(Y_test, y_prob_mi)
f1_mi = f1_score(Y_test, y_pred_mi)
cm_mi = confusion_matrix(Y_test, y_pred_mi)

print("Logistic Regression with MI-selected features (C tuned via 10-fold CV)")
print(f"AUC: {auc_mi:.4f}")
print(f"F1:  {f1_mi:.4f}")
print("Confusion matrix:")
print(cm_mi)
print("Chosen C:", logreg_mi_cv.C_)


Logistic Regression with MI-selected features (C tuned via 10-fold CV)
AUC: 0.7141
F1:  0.5145
Confusion matrix:
[[518 105]
 [297 213]]
Chosen C: [0.00077426]


In [22]:
# Compare LASSO vs MI Feature Selection
print("FEATURE SELECTION COMPARISON")
print("="*80)
print("\nLASSO Selected Features (12):")
print("-"*80)
for i, feature in enumerate(lasso_significant_variables, 1):
    print(f"{i:2d}. {feature}")

print("\n" + "="*80)
print("\nMI Selected Features (12):")
print("-"*80)
for i, feature in enumerate(mi_significant_variables, 1):
    print(f"{i:2d}. {feature}")

print("\n" + "="*80)
print("\nFeature Overlap Analysis:")
print("-"*80)
lasso_set = set(lasso_significant_variables)
mi_set = set(mi_significant_variables)
overlap = lasso_set.intersection(mi_set)
lasso_only = lasso_set - mi_set
mi_only = mi_set - lasso_set

print(f"Features in BOTH methods: {len(overlap)}")
for f in sorted(overlap):
    print(f"  ✓ {f}")

print(f"\nFeatures ONLY in LASSO: {len(lasso_only)}")
for f in sorted(lasso_only):
    print(f"  • {f}")

print(f"\nFeatures ONLY in MI: {len(mi_only)}")
for f in sorted(mi_only):
    print(f"  • {f}")


FEATURE SELECTION COMPARISON

LASSO Selected Features (12):
--------------------------------------------------------------------------------
 1. NBPd_mean
 2. Temp(F)_mean
 3. NBPm_mean
 4. Sodium_mean
 5. PlateletCount_mean
 6. RR_mean
 7. HR_mean
 8. SpO2_mean
 9. Hematocrit_mean
10. Chloride_mean
11. Urea_Nitrogren_mean
12. Glucose_mean


MI Selected Features (12):
--------------------------------------------------------------------------------
 1. Creatinine_mean
 2. Urea_Nitrogren_mean
 3. age_label
 4. Phosphate_mean
 5. Chloride_mean
 6. Calcium_Total_mean
 7. RR_mean
 8. SpO2_mean
 9. Hematocrit_mean
10. Glucose_mean
11. Magnesium_mean
12. Sodium_mean


Feature Overlap Analysis:
--------------------------------------------------------------------------------
Features in BOTH methods: 7
  ✓ Chloride_mean
  ✓ Glucose_mean
  ✓ Hematocrit_mean
  ✓ RR_mean
  ✓ Sodium_mean
  ✓ SpO2_mean
  ✓ Urea_Nitrogren_mean

Features ONLY in LASSO: 5
  • HR_mean
  • NBPd_mean
  • NBPm_mean
  • Pla

Barttender experiment with MI selected features

In [38]:
# Copying functions from barttender/mimic_add_bar_preprocess.py, replacing default bar_variables values with mi_significant_variables
import os
import fire
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import QuantileTransformer

from PIL import Image
from skimage.io import imread
from skimage.io import imsave, imshow
from skimage.transform import resize

def npy_bar(data, colormap, img_w=500, img_h=100, add_label=False, add_colormap=False, verbose=False):
    variables = data.columns[1:]  # Exclude ID
    variables = variables[::-1]  # Bars generated bottom to top

    for index, row in data.iterrows():
        px = 1/plt.rcParams['figure.dpi']  # pixel in inches
        bar_height = 1.0  # Height of each bar -- keep it to 1.0 to ensure no gap
        if verbose:
            fig, ax = plt.subplots(figsize=(10, 5))
        else:
            fig, ax = plt.subplots(figsize=(img_w*px, img_h*px))

        # Create horizontal bar plot
        for i, var in enumerate(variables):
            value = row[var]
            if np.isnan(value):
                color = 'r'  # i.e. 'red'
            else:
                color = colormap(value)
            ax.barh(i, 1, color=color, height=bar_height, edgecolor='none')

        # Set y-ticks and labels
        if add_label:
            ax.set_yticks(np.arange(len(variables)))
            ax.set_yticklabels(variables)
        else:
            ax.set_yticks([])

        # Remove grid and axes
        ax.grid(False)
        ax.set_xticks([])
        ax.set_xlim(0, 1)

        # Remove spines
        for spine in ax.spines.values():
            spine.set_visible(False)

        # Add colorbar
        if add_colormap:
            sm = plt.cm.ScalarMappable(
                cmap=colormap, norm=plt.Normalize(vmin=0, vmax=1))
            sm.set_array([])
            cbar = fig.colorbar(sm, ax=ax, fraction=0.025, pad=0.04)
            cbar.set_label('Normalized feature value')

        fig.subplots_adjust(left=0, right=1, top=1, bottom=0)
        fig.canvas.draw()
        if add_label or add_colormap:
            plt.show()
        else:
            bar_img = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
            bar_img = bar_img.reshape(fig.canvas.get_width_height()[::-1] + (4,))[:, :, :3]
            if img_h != 224:
                # slicing removes whitespace from bars; need to redo if bar size is changed
                bar_img = bar_img[3:-2, :, :]  # HWC
            plt.close(fig)
        
            if verbose:
                print(bar_img.shape)
                imshow(bar_img.astype(np.uint8))

            return bar_img

def preprocess_mimic_df(idp=False, order=None, bar_vars=mi_significant_variables, label='Cardiomegaly', verbose=False):
    variables = np.array(bar_vars)
    order, suffix = get_barcode_order_info(order, bar_vars)
    
    df_master = get_master_df(idp=idp)
    if not idp:
        df_master = df_master[df_master[label].isin([0, 1])]
        study_year = np.floor(df_master['StudyDate'] / 10000)
        delta_years = study_year - df_master['anchor_year']
        df_master['age'] = df_master['anchor_age'] + delta_years
    else:
        df_master['age'] = df_master['anchor_age']

        #adding age_label as a var since it's significant for MI selection
        df_master['age_label'] = df_master['age'] / 100
    
    # normalize age as in chexpert (0-100 to 0-1)
    df_master['age_val'] = df_master['age'].apply(lambda x: min(x / 100, 1))
    # quantile normalize the rest
    for var in bar_vars[1:]:
        transformer = QuantileTransformer(output_distribution='uniform')
        df_master.loc[:, var] = transformer.fit_transform(df_master[[var]])

    df = df_master[['path'] + bar_vars]

    if verbose:
        print(df.head())
        print(variables[order].tolist())
    
    return df[['path'] + variables[order].tolist()[0]]

def preprocess_and_append_bars(df, image_type, img_data_dir, idp=False, order=None, bar_vars=mi_significant_variables, colormap=plt.colormaps['binary'], verbose=False):
    df_cxr = df.copy()
    out_dir = img_data_dir

    order, suffix = get_barcode_order_info(order)
    # Hard coding shorter suffix to reduce file path length and increase clarity
    preproc_dir = get_preproc_subpath(image_type, 'MI_Vars')

    print(f'Image Type: {image_type}')
    print(f"Barcode Order: {suffix.replace('_', ', ')}")
    print(f'preproc_dir: {out_dir / preproc_dir}')
    
    bars_h = int(0.2 * 224) # 44
    whitespace_offset = 5 + 1 # num of pixel rows removed from bar output + 1 (row to remove because resizing artifact)

    if not verbose:
        os.makedirs(out_dir/ preproc_dir, exist_ok=True)

    for idx, p in enumerate(tqdm(df_cxr['path'])):
        out_path = construct_preproc_path(p, out_dir, preproc_dir)

        if (not os.path.exists(out_path)) or verbose:
            height = 224 + whitespace_offset - bars_h; width = 224
            if image_type == 'xray':
                image = imread(scratch_dir / p)  # og chexpert imagery is on scratch
                image = resize(image, output_shape=(height, width), preserve_range=True)
                image = np.expand_dims(image, axis=2).repeat(3, axis=2)
            elif image_type == 'noise':
                # Generate random noise
                image = np.random.randint(low=0, high=255, size=(height, width)) # 1-channel, B/W noise
                image = np.expand_dims(image, axis=2).repeat(3, axis=2)  # make it 3-channel
            elif image_type == 'blank':
                image = np.zeros(shape=(height, width, 3))  # all black image

            image = image[:-1,:,:]  # remove last img row (gets rid of resizing artifacts)

            # Generate the barcode and append it to the bottom of the image
            bar_img = npy_bar(df_cxr.iloc[[idx]], colormap, img_h=bars_h, img_w=width)
            
            combined_img = np.concatenate((image, bar_img), axis=0)
            
            if verbose:
                imshow(combined_img.astype(np.uint8))
            else:  
                imsave(out_path, combined_img.astype(np.uint8))

In [36]:
mi_significant_variables

['Creatinine_mean',
 'Urea_Nitrogren_mean',
 'age_label',
 'Phosphate_mean',
 'Chloride_mean',
 'Calcium_Total_mean',
 'RR_mean',
 'SpO2_mean',
 'Hematocrit_mean',
 'Glucose_mean',
 'Magnesium_mean',
 'Sodium_mean']

In [39]:
# Preprocess all with new 12 MI selected significant features
df_cxr_ex_2 = preprocess_mimic_df(idp=True, verbose=True, bar_vars=mi_significant_variables)

                                                path  Creatinine_mean  \
0  files/p10/p10001884/s55333410/5b135035-4708084...         0.905556   
1  files/p10/p10002013/s52163036/1e647043-eed3576...         1.000000   
2  files/p10/p10004457/s55439624/ad41d9ed-37ff140...         0.800000   
3  files/p10/p10011365/s56594284/ff295ca5-9f888b5...         0.587500   
4  files/p10/p10012292/s52412975/ca16ea39-8c22bfd...         0.766667   

   Urea_Nitrogren_mean  age_label  Phosphate_mean  Chloride_mean  \
0             0.575318   0.762763        0.306329       0.124027   
1             0.265265   0.283283             NaN       0.509510   
2             0.336722   0.505005        0.643644       0.544545   
3             0.269241   0.859359        0.046521       0.862863   
4             0.767768   0.656657        0.661662       0.936937   

   Calcium_Total_mean   RR_mean  SpO2_mean  Hematocrit_mean  Glucose_mean  \
0            0.774841  0.484323   0.726947         0.426632      0.773585  

In [40]:
# Image Barttender processing (xray + bars) 
preprocess_and_append_bars(
    df_cxr_ex_2,
    image_type='xray',
    img_data_dir=nb_group_dir,
    idp=True,
    bar_vars=lasso_significant_variables,
    verbose=False # IMPORTANT: it won't save the images if you run in verbose mode
)

Image Type: xray
Barcode Order: nbpd, temp(f), nbpm, sodium, plateletcount, rr, hr, spo2, hematocrit, chloride, urea, nitrogren, glucose
preproc_dir: physionet.org\files\mimic-cxr-jpg\2.1.0\preprocessed\preproc_224x224_xray_MI_Vars


100%|██████████| 2662/2662 [06:27<00:00,  6.88it/s]


In [41]:
# Rewriting functions from barttender/mimic_k_fold.py, replacing file path values as needed
import os
import fire
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
import torchvision
import torchvision.transforms as T
from torchvision import models
import pytorch_lightning as pl

from copy import deepcopy

import wandb
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from skimage.io import imread
from skimage.io import imsave
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix, f1_score, matthews_corrcoef

from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.tuner import Tuner
from sklearn.model_selection import KFold

image_size = (224, 224)
num_classes = 1

class MIMICDataset(Dataset):
    def __init__(self, csv_file_img, image_size, img_data_dir: Path, augmentation=False, pseudo_rgb=False):
        self.data = pd.read_csv(csv_file_img)
        self.image_size = image_size
        self.do_augment = augmentation
        self.pseudo_rgb = pseudo_rgb

        self.labels = ['Cardiomegaly']

        self.augment = T.Compose([
            T.RandomHorizontalFlip(p=0.5),
            T.RandomApply(transforms=[T.RandomAffine(degrees=15, scale=(0.9, 1.1))], p=0.5),
        ])

        self.samples = []
        for idx, _ in enumerate(tqdm(range(len(self.data)), desc='Loading Data')):
            og_path = self.data.loc[idx, 'path']
            img_path = construct_preproc_path(og_path, img_data_dir)
            img_label = np.array(self.data.loc[idx, self.labels[0].strip()] == 1, dtype='float32')

            sample = {'image_path': img_path, 'label': img_label}
            self.samples.append(sample)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        sample = self.get_sample(item)

        image = torch.from_numpy(sample['image'])
        # unsqueeze adds an extra dimension to the labels to match the output of the model probs
        label = torch.tensor(sample['label'], dtype=torch.float32).unsqueeze(0)

        if self.do_augment:
            image = self.augment(image)

        if self.pseudo_rgb:
            image = image.unsqueeze(0).repeat(3, 1, 1)
        else:
            image = image.permute(2, 0, 1)

        return {'image': image, 'label': label}

    def get_sample(self, item):
        sample = self.samples[item]
        image = imread(sample['image_path']).astype(np.float32)

        return {'image': image, 'label': sample['label']}

class MIMICDataModule(pl.LightningDataModule):
    def __init__(self, csv_train_img, csv_val_img, csv_test_img, image_size, img_data_dir: Path, pseudo_rgb, batch_size, num_workers):
        super().__init__()
        self.csv_train_img = csv_train_img
        self.csv_val_img = csv_val_img
        self.csv_test_img = csv_test_img
        self.image_size = image_size
        self.batch_size = batch_size
        self.num_workers = num_workers

        self.train_set = MIMICDataset(self.csv_train_img, self.image_size, img_data_dir, augmentation=True, pseudo_rgb=pseudo_rgb)
        self.val_set = MIMICDataset(self.csv_val_img, self.image_size, img_data_dir, augmentation=False, pseudo_rgb=pseudo_rgb)
        self.test_set = MIMICDataset(self.csv_test_img, self.image_size, img_data_dir,  augmentation=False, pseudo_rgb=pseudo_rgb)

        print('#train: ', len(self.train_set))
        print('#val:   ', len(self.val_set))
        print('#test:  ', len(self.test_set))

    def train_dataloader(self):
        return DataLoader(self.train_set, self.batch_size, shuffle=True, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_set, self.batch_size, shuffle=False, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test_set, self.batch_size, shuffle=False, num_workers=self.num_workers)


class ResNet(pl.LightningModule):
    def __init__(self, num_classes):
        super().__init__()
        self.num_classes = num_classes
        self.model = models.resnet34(pretrained=True)
        # freeze_model(self.model)
        num_features = self.model.fc.in_features
        self.model.fc = nn.Linear(num_features, self.num_classes)

    def remove_head(self):
        num_features = self.model.fc.in_features
        id_layer = nn.Identity(num_features)
        self.model.fc = id_layer

    def forward(self, x):
        return self.model.forward(x)

    def configure_optimizers(self):
        params_to_update = []
        for param in self.parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
        optimizer = torch.optim.Adam(params_to_update, lr=0.001)
        return optimizer

    def unpack_batch(self, batch):
        return batch['image'], batch['label']

    def process_batch(self, batch):
        img, lab = self.unpack_batch(batch)
        out = self.forward(img)
        prob = torch.sigmoid(out)
        loss = F.binary_cross_entropy(prob, lab)
        return loss

    def training_step(self, batch, batch_idx):
        loss = self.process_batch(batch)
        self.log('train_loss', loss)
        grid = torchvision.utils.make_grid(batch['image'][0:4, ...], nrow=2, normalize=True)
        self.logger.experiment.add_image('images', grid, self.global_step)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.process_batch(batch)
        self.log('val_loss', loss, sync_dist=True)

    def test_step(self, batch, batch_idx):
        loss = self.process_batch(batch)
        self.log('test_loss', loss)


class DenseNetTrain(pl.LightningModule):
    def __init__(self, num_classes):
        super().__init__()
        self.save_hyperparameters()
        self.num_classes = num_classes
        self.model = models.densenet121(pretrained=True)
        # freeze_model(self.model)
        num_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(num_features, self.num_classes)

    def remove_head(self):
        num_features = self.model.classifier.in_features
        id_layer = nn.Identity(num_features)
        self.model.classifier = id_layer

    def forward(self, x):
        return self.model.forward(x)

    def configure_optimizers(self):
        params_to_update = []
        for param in self.parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
        optimizer = torch.optim.Adam(params_to_update, lr=0.001)
        return optimizer

    def unpack_batch(self, batch):
        return batch['image'], batch['label']

    def process_batch(self, batch):
        img, lab = self.unpack_batch(batch)
        out = self.forward(img)
        prob = torch.sigmoid(out)
        loss = F.binary_cross_entropy(prob, lab)
        return loss

    def training_step(self, batch, batch_idx):
        loss = self.process_batch(batch)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.process_batch(batch)
        self.log('val_loss', loss, sync_dist=True)

    def test_step(self, batch, batch_idx):
        loss = self.process_batch(batch)
        self.log('test_loss', loss)


def freeze_model(model):
    for param in model.parameters():
        param.requires_grad = False


def test(model, data_loader, device):
    model.eval()
    logits = []
    preds = []
    targets = []

    with torch.no_grad():
        for index, batch in enumerate(tqdm(data_loader, desc='Test-loop')):
            img, lab = batch['image'].to(device), batch['label'].to(device)
            out = model(img)
            pred = torch.sigmoid(out)
            logits.append(out)
            preds.append(pred)
            targets.append(lab)

        logits = torch.cat(logits, dim=0)
        preds = torch.cat(preds, dim=0)
        targets = torch.cat(targets, dim=0)

        preds_np = preds.cpu().numpy()
        targets_np = targets.cpu().numpy()

        auc_scores = []
        for i in range(num_classes):
            auc = roc_auc_score(targets_np[:, i], preds_np[:, i])
            auc_scores.append(auc)
        
        for i, auc in enumerate(auc_scores):
            print(f"Cardiomegaly AUC: {auc:.4f}")

    return preds_np, targets_np, logits.cpu().numpy(), auc_scores

def embeddings(model, data_loader, device):
    model.eval()

    embeds = []
    targets = []

    with torch.no_grad():
        for index, batch in enumerate(tqdm(data_loader, desc='Test-loop')):
            img, lab = batch['image'].to(device), batch['label'].to(device)
            emb = model(img)
            embeds.append(emb)
            targets.append(lab)

        embeds = torch.cat(embeds, dim=0)
        targets = torch.cat(targets, dim=0)

    return embeds.cpu().numpy(), targets.cpu().numpy()

def compute_metrics(y_true, y_pred, y_logits):
    """Compute various classification metrics for binary classification."""
    # y_true: Actual binary labels (0 or 1)
    # y_pred: Predicted binary labels (0 or 1), thresholding applied
    # y_logits: Raw model outputs (logits) or probabilities for the positive class

    # Convert y_logits to probabilities if they are logits
    y_prob = y_logits if y_logits.shape[1] == 1 else y_logits[:, 1]

    auc = roc_auc_score(y_true, y_prob)
    avg_precision = average_precision_score(y_true, y_prob)
    
    # Convert y_pred to binary predictions if it's probabilities
    y_pred_binary = (y_pred > 0.5).astype(int)
    
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred_binary)
    
    # Metrics calculations
    tpr = cm[1, 1] / (cm[1, 1] + cm[1, 0]) if (cm[1, 1] + cm[1, 0]) > 0 else 0  # Sensitivity, Recall
    tnr = cm[0, 0] / (cm[0, 0] + cm[0, 1]) if (cm[0, 0] + cm[0, 1]) > 0 else 0  # Specificity
    ppv = cm[1, 1] / (cm[1, 1] + cm[0, 1]) if (cm[1, 1] + cm[0, 1]) > 0 else 0  # Precision
    npv = cm[0, 0] / (cm[0, 0] + cm[1, 0]) if (cm[0, 0] + cm[1, 0]) > 0 else 0  # Negative Predictive Value
    f1 = f1_score(y_true, y_pred_binary)
    mcc = matthews_corrcoef(y_true, y_pred_binary)

    return {
        'AUC': auc,
        'Average Precision': avg_precision,
        'TPR': tpr,
        'TNR': tnr,
        'PPV': ppv,
        'NPV': npv,
        'F1': f1,
        'MCC': mcc
    }


def main(image_type: str = 'xray', order = None, batch_size: int = 192, epochs: int = 100, num_workers: int = 0, n_splits: int = 10, idp: bool = True, nan: bool = False, only_bars: bool = False, no_bars: bool = False):

    # sets seeds for numpy, torch, python.random and PYTHONHASHSEED.
    pl.seed_everything(42, workers=True)
    image_type = 'only_bars' if only_bars else image_type
    order, suffix = get_barcode_order_info(order=order, no_bars=no_bars, nan=nan, only_bars=only_bars)
    preproc_dir = get_preproc_subpath(image_type, "MI_Vars")
    root_dir = get_correct_root_dir(preproc_dir)
    img_data_dir = root_dir / preproc_dir

    # Create output directory for this cross-validation experiment
    # Hardcoding - replacing suffix with MI_Vars
    exp_dir = home_out_dir / f"cross-val/densenet-{image_type}-{'MI_Vars'}-{'idp' if idp else 'no_idp'}/"
    if not os.path.exists(exp_dir):
        os.makedirs(exp_dir)
    df = get_cardiomegaly_df(idp=idp)
    if idp: df['split'] = 'train'
    train_df, val_df, test_df = df_train_test_split(df, test_size=0.2, val_size=0.1)
    train_df.to_csv(exp_dir / 'train.csv')    
    val_df.to_csv(exp_dir / 'val.csv')
    test_df.to_csv(exp_dir / 'test.csv')
    
    data = MIMICDataModule(csv_train_img=(exp_dir / 'train.csv').as_posix(),
                              csv_val_img=(exp_dir / 'val.csv').as_posix(),
                              csv_test_img=(exp_dir / 'test.csv').as_posix(),
                              img_data_dir=img_data_dir,
                              image_size=image_size,
                              pseudo_rgb=False,
                              batch_size=batch_size,
                              num_workers=num_workers)

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Determine GPU usage and training strategy
    use_cuda = torch.cuda.is_available()
    num_gpus = torch.cuda.device_count()
    accelerator = "gpu" if use_cuda else None
    device = "cuda" if use_cuda else "cpu"
    strategy = "ddp" if num_gpus > 1 else "auto"

    # Optimize batch size if only one GPU is available
    if num_gpus == 1 and False:
        model_type = DenseNetTrain(num_classes=num_classes)
        trainer = pl.Trainer(accelerator=accelerator, devices=1)
        tuner = Tuner(trainer)
        optimal_batch_size = tuner.scale_batch_size(model_type, datamodule=data)
        data.batch_size = optimal_batch_size
        print(f'Optimal batch size found: {optimal_batch_size}')
    else:
        optimal_batch_size = batch_size
        print(f'Optimal batch size found: {optimal_batch_size}')

    all_val_metrics = []
    all_test_metrics = []
    fold_metrics = []
    fold_data = deepcopy(data)
    for fold, (train_idx, val_idx) in enumerate(kf.split(data.train_set)):
        print(f'\n--- Fold {fold+1}/{n_splits} ---')

        # Split data for this fold
        train_subset = torch.utils.data.Subset(data.train_set, train_idx)
        val_subset = torch.utils.data.Subset(data.train_set, val_idx)
        
        fold_data.train_set = train_subset
        fold_data.val_set = val_subset

        if max(train_idx) >= len(data.train_set) or max(val_idx) >= len(data.train_set):
            raise IndexError("Indices out of bounds.")

        # Model
        model_type = DenseNetTrain
        model = model_type(num_classes=num_classes)

        checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode='min')
        early_stopping_callback = EarlyStopping(monitor="val_loss", mode='min', patience=5, verbose=True)

        config = {}
        config["pseudo_rgb"] = False
        config["batch_size"] = optimal_batch_size
        config["image_size"] = image_size
        config["image_type"] = image_type
        config["barcode_order"] = suffix
        config["img_data_dir"] = img_data_dir
        config["idp"] = idp
        config['csv_train'] = (exp_dir / 'train.csv').as_posix()
        config['csv_val']   = (exp_dir / 'val.csv').as_posix()
        config['csv_test']  = (exp_dir / 'test.csv').as_posix()

        # Logger for this fold
        wandb_logger = WandbLogger(project='mimic', config=config, save_dir=home_out_dir)

        # Create output directory for this fold
        out_name = f"fold{fold+1}-{wandb_logger.experiment.id}/"
        out_dir = exp_dir / out_name
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)
        wandb_logger.experiment.config["out_dir"] = out_dir

        # Trainer
        trainer = pl.Trainer(
            callbacks=[checkpoint_callback, early_stopping_callback],
            log_every_n_steps=5,
            max_epochs=epochs,
            logger=wandb_logger,
            accelerator=accelerator,
            devices="auto" if use_cuda else 1,
            num_nodes=1,
            strategy=strategy
        )
        trainer.logger._default_hp_metric = False

        # Train the model
        trainer.fit(model, fold_data)

        # Load the best model from this fold
        model = model_type.load_from_checkpoint(trainer.checkpoint_callback.best_model_path, num_classes=num_classes)
        model.to(device)

        # Validation
        print(f'\nVALIDATION for Fold {fold+1}')
        preds_val, targets_val, logits_val, aucs_val = test(model, fold_data.val_dataloader(), device)
        val_metrics = compute_metrics(targets_val, preds_val, logits_val)
        all_val_metrics.append(val_metrics)

        # Testing
        print(f'\nTESTING for Fold {fold+1}')
        preds_test, targets_test, logits_test, aucs_test = test(model, fold_data.test_dataloader(), device)
        test_metrics = compute_metrics(targets_test, preds_test, logits_test)
        all_test_metrics.append(test_metrics)

        # Log metrics for this fold
        for metric_name, metric_value in val_metrics.items():
            wandb_logger.log_metrics({f'fold_{fold+1}_val_{metric_name.lower()}': metric_value})
        for metric_name, metric_value in test_metrics.items():
            wandb_logger.log_metrics({f'fold_{fold+1}_test_{metric_name.lower()}': metric_value})

        cols_names_classes = ['class_' + str(i) for i in range(0,num_classes)]
        cols_names_logits = ['logit_' + str(i) for i in range(0, num_classes)]
        cols_names_targets = ['target_' + str(i) for i in range(0, num_classes)]

        # Save validation and test predictions for this fold
        df_val = pd.DataFrame(data=preds_val, columns=cols_names_classes)
        df_val_logits = pd.DataFrame(data=logits_val, columns=cols_names_logits)
        df_val_targets = pd.DataFrame(data=targets_val, columns=cols_names_targets)
        df_val = pd.concat([df_val, df_val_logits, df_val_targets], axis=1)
        df_val.to_csv(os.path.join(out_dir, 'predictions.val.csv'), index=False)

        df_test = pd.DataFrame(data=preds_test, columns=cols_names_classes)
        df_test_logits = pd.DataFrame(data=logits_test, columns=cols_names_logits)
        df_test_targets = pd.DataFrame(data=targets_test, columns=cols_names_targets)
        df_test = pd.concat([df_test, df_test_logits, df_test_targets], axis=1)
        df_test.to_csv(os.path.join(out_dir, 'predictions.test.csv'), index=False)

        # Store fold-wise metrics
        fold_metrics.append({
            'fold': fold+1,
            'val_metrics': val_metrics,
            'test_metrics': test_metrics
        })

        wandb.finish()

    # Aggregate metrics over all folds
    val_metrics_mean = {key: np.mean([m[key] for m in all_val_metrics]) for key in all_val_metrics[0].keys()}
    test_metrics_mean = {key: np.mean([m[key] for m in all_test_metrics]) for key in all_test_metrics[0].keys()}
    val_metrics_std = {key: np.std([m[key] for m in all_val_metrics]) for key in all_val_metrics[0].keys()}
    test_metrics_std = {key: np.std([m[key] for m in all_test_metrics]) for key in all_test_metrics[0].keys()}
    best_folds = {key: max(enumerate(all_val_metrics), key=lambda x: x[1][key])[0]+1 for key in all_val_metrics[0].keys()}

    print(f'\nMean Validation Metrics across {n_splits} folds: {val_metrics_mean}')
    print(f'\nMean Test Metrics across {n_splits} folds: {test_metrics_mean}')

    # Save overall and fold-wise metrics to CSV
    metrics_summary = {
        'metric': list(val_metrics_mean.keys()),
        'val_mean': list(val_metrics_mean.values()),
        'val_std': list(val_metrics_std.values()),
        'test_mean': list(test_metrics_mean.values()),
        'test_std': list(test_metrics_std.values()),
        'best_fold': [best_folds[key] for key in val_metrics_mean.keys()]
    }
    df_metrics_summary = pd.DataFrame(metrics_summary)
    df_metrics_summary.to_csv(exp_dir / 'metrics_summary.csv', index=False)

    df_fold_metrics = pd.DataFrame(fold_metrics)
    val_metrics_df = df_fold_metrics['val_metrics'].apply(pd.Series).add_prefix('val_')
    test_metrics_df = df_fold_metrics['test_metrics'].apply(pd.Series).add_prefix('test_')

    # Concatenate the original DataFrame with the new metrics columns
    df_fold_metrics = pd.concat([df_fold_metrics[['fold']], val_metrics_df, test_metrics_df], axis=1)
    df_fold_metrics.to_csv(exp_dir / 'fold_metrics.csv', index=False)

    # Log the fold that was best for each metric
    print(f'\nBest folds based on validation metrics: {best_folds}')

Training DenseNet model with MI selected vars

In [42]:
import importlib
import os
import wandb
os.environ['WANDB_MODE'] = 'offline'

main(
    image_type = 'xray',
    batch_size=32,
    epochs=100,
    n_splits=10,
    idp=True,
    num_workers=0
)

Seed set to 42
Loading Data: 100%|██████████| 537/537 [00:00<00:00, 89520.72it/s]


#train:  1849
#val:    276
#test:   537
Optimal batch size found: 32

--- Fold 1/10 ---


c:\Users\Josh\Desktop\DLH\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Josh\Desktop\DLH\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
wandb: WARNING `resume` will be ignored since W&B syncing is set to `offline`. Starting a new run with run id 3q33etvb.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
You are using a CUDA device ('NVIDIA GeForce RTX 4070') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type     | Params | Mode  | FLOPs
---------------------------------------------------
0 | model | DenseNet | 7.0 M  | train | 0    
---------------------------------------------------
7.0 M     Trainable params
0         Non-trainable params
7.0 M     Total params
27.820    Total estimated model params size (MB)
433       Modules in train mode
0         Modules in eval mode
0         Total Flops


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Test-loop: 100%|██████████| 17/17 [00:00<00:00, 18.05it/s]


Cardiomegaly AUC: 0.8282


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇▇███████
fold_10_test_auc,▁
fold_10_test_average precision,▁
fold_10_test_f1,▁
fold_10_test_mcc,▁
fold_10_test_npv,▁
fold_10_test_ppv,▁
fold_10_test_tnr,▁
fold_10_test_tpr,▁
fold_10_val_auc,▁
+10,...



Mean Validation Metrics across 10 folds: {'AUC': np.float64(0.7679544928676194), 'Average Precision': np.float64(0.8774441596995967), 'TPR': np.float64(0.8895243499741156), 'TNR': np.float64(0.43714819047042164), 'PPV': np.float64(0.7861490865175875), 'NPV': np.float64(0.6407546823126626), 'F1': np.float64(0.8333140682561627), 'MCC': np.float64(0.371376288101777)}

Mean Test Metrics across 10 folds: {'AUC': np.float64(0.8582443633952254), 'Average Precision': np.float64(0.9294293806635927), 'TPR': np.float64(0.8116710875331565), 'TNR': np.float64(0.7056250000000001), 'PPV': np.float64(0.8727600917422913), 'NPV': np.float64(0.6352501780114876), 'F1': np.float64(0.8368442755440955), 'MCC': np.float64(0.509674880553376)}

Best folds based on validation metrics: {'AUC': 9, 'Average Precision': 9, 'TPR': 4, 'TNR': 9, 'PPV': 9, 'NPV': 6, 'F1': 4, 'MCC': 6}


Comparing Image barttender metrics between main experiment replication and both extensions

In [ ]:
experiments = [
    "densenet-xray-age_chloride_rr_urea_nitrogren_magnesium_glucose_phosphate_hematocrit-idp",
    "densenet-xray-LASSO_Vars-idp",
    "densenet-xray-MI_Vars-idp"
]


In [47]:
# Main barttender experiment output
print(pd.read_csv("physionet.org/outputs/cardiomegaly/cross-val/densenet-xray-age_chloride_rr_urea_nitrogren_magnesium_glucose_phosphate_hematocrit-idp/metrics_summary.csv"))

              metric  val_mean   val_std  test_mean  test_std  best_fold
0                AUC  0.795241  0.036511   0.850225  0.033183          2
1  Average Precision  0.896415  0.030586   0.924384  0.021033          2
2                TPR  0.921207  0.045002   0.845093  0.083940          8
3                TNR  0.397963  0.129313   0.656250  0.130922          2
4                PPV  0.779063  0.051599   0.857315  0.036641          2
5                NPV  0.703808  0.099375   0.666784  0.084608          1
6                 F1  0.842705  0.035502   0.847126  0.032960          2
7                MCC  0.387390  0.111908   0.510334  0.054826          2


In [48]:
# LASSO Selected Variable Extension
print(pd.read_csv("physionet.org/outputs/cardiomegaly/cross-val/densenet-xray-LASSO_Vars-idp/metrics_summary.csv"))

              metric  val_mean   val_std  test_mean  test_std  best_fold
0                AUC  0.790508  0.060913   0.864559  0.014968          1
1  Average Precision  0.894781  0.036702   0.933545  0.007613          9
2                TPR  0.908010  0.026744   0.858886  0.048575          7
3                TNR  0.453192  0.107721   0.675000  0.129120          9
4                PPV  0.793535  0.041322   0.865373  0.039834          9
5                NPV  0.680839  0.082607   0.678389  0.049754          7
6                 F1  0.845821  0.019422   0.860062  0.014585          4
7                MCC  0.412067  0.081351   0.537190  0.063605          9


In [49]:
# MI Selected Variable Extension
print(pd.read_csv("physionet.org/outputs/cardiomegaly/cross-val/densenet-xray-MI_Vars-idp/metrics_summary.csv"))

              metric  val_mean   val_std  test_mean  test_std  best_fold
0                AUC  0.767954  0.045918   0.858244  0.018543          9
1  Average Precision  0.877444  0.032193   0.929429  0.013991          9
2                TPR  0.889524  0.045229   0.811671  0.078633          4
3                TNR  0.437148  0.111134   0.705625  0.148993          9
4                PPV  0.786149  0.037978   0.872760  0.042578          9
5                NPV  0.640755  0.070092   0.635250  0.076936          6
6                 F1  0.833314  0.024504   0.836844  0.024073          4
7                MCC  0.371376  0.071750   0.509675  0.039093          6
